In [1]:
# -*- coding: utf-8 -*-
import collections
from naver_ai import NaverAI 

naver = NaverAI()

In [2]:
request_data = {
    "maxTokens": 32,
    "temperature": 0.5,
    "topK": 0,
    "topP": 0.8,
    "repeatPenalty": 5.0,
    "restart": "",
    "includeTokens": True,
    "includeAiFilters": True,
    "includeProbs": False
}

In [9]:
keys = ["text", "start", "stopBefore", "restart"]

def make_content(original_dict: dict, keys: list=keys, *args):
    text_gen_req = {key: value for key, value in zip(keys, *args)}
    return dict(collections.ChainMap(original_dict, text_gen_req))

## Text Generation

In [10]:
preset_text = "한국어: 사과\n영어: Apple\n한국어: 바나나"
start = "\n영어"
stopBefore = ["\n한국어"]

text_gen_req = make_content(request_data, keys, [preset_text, start, stopBefore])

In [11]:
generation_result = naver.execute(text_gen_req)
generation_result

{'status': {'code': '20000', 'message': 'OK', 'messageVariables': {}},
 'result': {'aiFilter': [{'groupName': 'curse',
    'name': 'insult',
    'score': '2'},
   {'groupName': 'curse', 'name': 'discrimination', 'score': '2'},
   {'groupName': 'unsafeContents', 'name': 'sexualHarassment', 'score': '2'}],
  'text': '한국어: 사과\n영어: Apple\n한국어: 바나나\n영어: Banana',
  'stopReason': 'stop_before',
  'inputLength': 15,
  'outputLength': 7,
  'inputTokens': ['한국',
   '어',
   ':',
   ' 사과',
   '\n',
   '영어',
   ':',
   ' Apple',
   '\n',
   '한국',
   '어',
   ':',
   ' 바나나',
   '\n',
   '영어'],
  'outputTokens': [':', ' B', 'an', 'ana', '\n', '한국', '어'],
  'probs': [],
  'ok': True}}

## Transformation

In [12]:
preset_text = '입력한 문장의 틀린 문법을 올바른 문장으로 변환합니다.\n###\n문장: 너랑 걔네들은 다르잖아. 너대로 하면되\n교정: 너랑 걔네들은 다르잖아. 너대로 하면 돼\n###\n문장: 이런식으로 행동하면 어떻게?\n교정: 이런 식으로 행동하면 어떡해?\n###\n문장: 이제 됬어요?\n교정: 이제 됐어요?\n###\n문장: 않돼 그러지마\n교정: 안 돼 그러지 마\n###\n문장: 그건 동물이예요\n'
start = "\n교정"
restart = "\n###\n문장:"
stopBefore = ['###', '문장:', '교정:', '###\n']

In [13]:
transform_req = make_content(request_data, keys, [preset_text, start, stopBefore, restart])

In [16]:
transform_result = naver.execute(transform_req)
transform_result

{'status': {'code': '20000', 'message': 'OK', 'messageVariables': {}},
 'result': {'aiFilter': [{'groupName': 'curse',
    'name': 'insult',
    'score': '2'},
   {'groupName': 'curse', 'name': 'discrimination', 'score': '2'},
   {'groupName': 'unsafeContents', 'name': 'sexualHarassment', 'score': '2'}],
  'text': '입력한 문장의 틀린 문법을 올바른 문장으로 변환합니다.\n###\n문장: 너랑 걔네들은 다르잖아. 너대로 하면되\n교정: 너랑 걔네들은 다르잖아. 너대로 하면 돼\n###\n문장: 이런식으로 행동하면 어떻게?\n교정: 이런 식으로 행동하면 어떡해?\n###\n문장: 이제 됬어요?\n교정: 이제 됐어요?\n###\n문장: 않돼 그러지마\n교정: 안 돼 그러지 마\n###\n문장: 그건 동물이예요\n\n교정: 그건 동물이에요\n',
  'stopReason': 'stop_before',
  'inputLength': 115,
  'outputLength': 7,
  'inputTokens': ['입력',
   '한',
   ' 문장',
   '의',
   ' 틀린',
   ' 문법',
   '을',
   ' 올바른',
   ' 문장',
   '으로',
   ' 변환',
   '합니다',
   '.',
   '\n',
   '##',
   '#',
   '\n',
   '문장',
   ':',
   ' 너',
   '랑',
   ' 걔',
   '네',
   '들은',
   ' 다르',
   '잖아',
   '.',
   ' 너',
   '대로',
   ' 하면',
   '되',
   '\n',
   '교정',
   ':',
   ' 너',
   '랑',
   ' 걔',
   '네',
   '들은',
   '